In [1]:
import os
import pickle
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import interact

# --- Config ---
data_path = "../data"
instance = "instAD2"
distance_method = "haversine"
optimization_variable = "hybrid"  # Or "distance", "extra_time", etc.

# --- Load the trace and best results ---
trace_path = f"{data_path}/resultados/alpha_calibration/{instance}/{distance_method}/{optimization_variable}_trace.csv"
best_path = f"{data_path}/resultados/alpha_calibration/{instance}/{distance_method}/{optimization_variable}_best.pkl"

trace_df = pd.read_csv(trace_path)
with open(best_path, "rb") as f:
    best_results = pickle.load(f)

print("✅ Loaded trace data:", trace_df.shape)
print("✅ Loaded best results:", len(best_results))


✅ Loaded trace data: (10241, 11)
✅ Loaded best results: 49


In [ ]:
# trace_df = trace_df.drop(columns=['instance', 'distance_method', 'objective_value', 'optimization_variable'])

In [ ]:
def plot_alpha_calibration(trace_df):
    def _plot(city, alpha, metric):
        df = trace_df[(trace_df["city"] == city) & (trace_df["alpha"] == alpha)]
        if df.empty:
            print("⚠️ No data for this selection.")
            return

        fig = px.line(
            df,
            x="iteration",
            y=metric,
            markers=True,
            title=f"City: {city} | Alpha: {alpha}",
            template="plotly_white"
        )

        fig.update_layout(
            xaxis_title="Iteration",
            yaxis_title=metric,
            legend_title="Metric"
        )
        fig.show()

    interact(
        _plot,
        city=widgets.Dropdown(
            options=sorted(trace_df["city"].unique()),
            description="City:"
        ),
        alpha=widgets.Dropdown(
            options=sorted(trace_df["alpha"].unique()),
            description="Alpha:"
        ),
        metric=widgets.Dropdown(
            options=["vt_labors", "extra_time", "dist"],
            description="Metric:"
        )
    )

plot_alpha_calibration(trace_df)


interactive(children=(Dropdown(description='City:', options=(np.int64(1), np.int64(126), np.int64(149), np.int…

In [2]:
trace_df[(trace_df['city']==1) & (trace_df['alpha']==0)]

,instance,distance_method,optimization_variable,city,date,alpha,iteration,objective_value,vt_labors,extra_time,dist
19,instAD2,haversine,hybrid,1,2026-01-05,0.0,52,26,26,423.7,63.21
20,instAD2,haversine,hybrid,1,2026-01-05,0.0,105,26,26,423.7,63.21
21,instAD2,haversine,hybrid,1,2026-01-05,0.0,157,26,26,423.7,63.21
22,instAD2,haversine,hybrid,1,2026-01-05,0.0,210,26,26,423.7,63.21
23,instAD2,haversine,hybrid,1,2026-01-05,0.0,263,26,26,423.7,63.21
...,...,...,...,...,...,...,...,...,...,...,...
831,instAD2,haversine,hybrid,1,2026-01-11,0.0,789,26,26,636.8,80.47
832,instAD2,haversine,hybrid,1,2026-01-11,0.0,842,26,26,636.8,80.47
833,instAD2,haversine,hybrid,1,2026-01-11,0.0,894,26,26,636.8,80.47
834,instAD2,haversine,hybrid,1,2026-01-11,0.0,947,26,26,636.8,80.47


In [8]:
agg_df = (
    trace_df.groupby(["city", "alpha"])
    .agg({
        "vt_labors": "max",
        "extra_time": "mean",
        "dist": "mean"
    })
    .reset_index()
)

fig = px.line(
    agg_df,
    x="alpha",
    y="objective_value",
    color="city",
    markers=True,
    title="Objective Value per Alpha by City",
    template="plotly_white"
)
fig.show()


ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['city', 'alpha', 'vt_labors', 'extra_time', 'dist'] but received: objective_value

In [5]:
fig = px.line(
    trace_df,
    x="iteration",
    y="objective_value",
    color="alpha",
    facet_col="city",
    title="Incumbent Objective Evolution per Alpha and City",
    template="plotly_white"
)
fig.show()


In [6]:
best_summary = []
for (city, date), info in best_results.items():
    best_summary.append({
        "city": city,
        "date": date,
        "alpha": info["alpha"],
        "vt_labors": info["vt_labors"],
        "extra_time": info["extra_time"],
        "dist": info["dist"]
    })
best_df = pd.DataFrame(best_summary)
display(best_df.head())


,city,date,alpha,vt_labors,extra_time,dist
0,149,2026-01-05,0.1,73,1291.2,214.29
1,1,2026-01-05,0.2,26,363.7,67.66
2,126,2026-01-05,0.0,8,268.0,25.35
3,150,2026-01-05,0.0,4,30.0,12.51
4,844,2026-01-05,0.0,5,144.1,29.72


In [7]:
fig = px.scatter(
    best_df,
    x="alpha",
    y="dist",
    color="city",
    size="vt_labors",
    hover_data=["extra_time"],
    title="Best Distances vs Alpha",
    template="plotly_white"
)
fig.show()
